## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-02-20-00 +0000,nyt,Judge Rules Trump Exceeded Authority by Holdin...,https://www.nytimes.com/2025/12/05/us/politics...
1,2025-12-06-02-04-36 +0000,bbc,Legendary architect Frank Gehry dies aged 96,https://www.bbc.com/news/articles/c5y2p22z9gno...
2,2025-12-06-02-03-29 +0000,bbc,US and Ukraine negotiators say 'real progress'...,https://www.bbc.com/news/articles/clydxj7my84o...
3,2025-12-06-02-00-00 +0000,wsj,Takeaways from Robert F. Kennedy Jr.’s vaccine...,https://www.wsj.com/politics/skeptics-in-the-s...
4,2025-12-06-01-59-08 +0000,latimes,Pet owners find 'grisly' scene inside San Marc...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,46
253,netflix,15
255,warner,14
486,court,14
77,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
172,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...,104
259,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...,102
139,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,101
213,2025-12-05-15-33-01 +0000,nypost,Trump national security strategy calls out ‘an...,https://nypost.com/2025/12/05/us-news/trump-na...,95
77,2025-12-05-22-05-50 +0000,nyt,Supreme Court Agrees to Review Trump Order Res...,https://www.nytimes.com/2025/12/05/us/politics...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
172,104,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...
247,81,2025-12-05-12-33-52 +0000,latimes,Netflix agrees to buy Warner Bros. in an $82.7...,https://www.latimes.com/entertainment-arts/bus...
190,63,2025-12-05-16-51-04 +0000,nypost,DC pipe-bomb suspect Brian Cole Jr. appears in...,https://nypost.com/2025/12/05/us-news/accused-...
173,43,2025-12-05-17-46-00 +0000,wsj,Having cast the deciding vote to secure Robert...,https://www.wsj.com/politics/policy/rfk-cdc-bi...
191,43,2025-12-05-16-44-00 +0000,wsj,New York Times Escalates Battle Against Perple...,https://www.wsj.com/business/media/new-york-ti...
88,40,2025-12-05-21-47-07 +0000,nyt,Video of Boat Strike Shows Survivors Waving Be...,https://www.nytimes.com/2025/12/05/us/politics...
213,39,2025-12-05-15-33-01 +0000,nypost,Trump national security strategy calls out ‘an...,https://nypost.com/2025/12/05/us-news/trump-na...
259,34,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...
212,33,2025-12-05-15-35-40 +0000,bbc,'I don't want to be part of this war machine':...,https://www.bbc.com/news/videos/cx25031x23ro?a...
109,33,2025-12-05-20-48-41 +0000,nypost,Ex-NYC Council staffers threatened with legal ...,https://nypost.com/2025/12/05/us-news/favorite...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
